# Py City School District Analysis

In [1]:
import pandas as pd
school_data = pd.read_csv('Resources/schools_complete.csv')
student_data = pd.read_csv('Resources/students_complete.csv')

complete_school_data = pd.merge(student_data, school_data, how="left", on="school_name")
# complete_school_data.head() - verified merge

## Creating District Summary Table

In [2]:
#Calculating number of schools:
schools = len(school_data.school_name)

#Calulating number of students and storing a formatted version for the table:
students = student_data.student_name.count()
students_formatted = '{:,.0f}'.format(students)

#Calulating budget and storing a formatted version for the table:
budget = school_data.budget.sum()
budget_formatted = '${:,.2f}'.format(float(budget))

#Calculating average math and reading scores:
avg_math = round(student_data.math_score.mean(), 6)
avg_reading = round(student_data.reading_score.mean(),6)

#Verified outputs
#print(schools, students, students_formatted, budget, budget_formatted, avg_math, avg_reading)

In [3]:
#Calculating passing percentages:
pct_pass_math = (len(student_data.loc[(student_data['math_score']>=70)])/students)*100
pct_pass_reading = (len(student_data.loc[(student_data['reading_score']>=70)])/students)*100
pct_pass_combined = (len(student_data.loc[(student_data['math_score']>=70) & 
                                                (student_data['reading_score']>=70)])/students)*100
#Verified outputs
#print(pct_pass_math, pct_pass_reading, pct_pass_combined)

In [4]:
#Building Data Frame
district_summary = pd.DataFrame({'Total Schools':[schools],
                                 'Total Students': [students_formatted],
                                 'Total Budget': [budget_formatted],
                                 'Average Math Score': [avg_math],
                                 'Average Reading Score': [avg_reading],
                                 '% Passing Math': [pct_pass_math],
                                 '% Passing Reading': [pct_pass_reading],
                                 '% Overall Passing': [pct_pass_combined]
                                })
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## Creating School Summary Table

In [5]:
# Using school_data as starting point for school summary table
school_summary = school_data.copy()
school_summary.drop('School ID', axis=1, inplace=True)
school_summary.set_index('school_name', inplace=True)
school_summary.sort_index(inplace=True)
# school_summary verified output

In [6]:
# Calculating Budget per Student and adding column
school_summary['Budget per Student'] = school_summary.budget/school_summary['size']
# school_summary verified output

In [7]:
#Calculating Average Math and Reading Scores and adding concatenating the two DFs
avg_scores = complete_school_data.groupby('school_name').agg({'math_score': ['mean'],
                                                            'reading_score': ['mean'],
                                                            })
school_summary = pd.concat([school_summary, avg_scores], axis=1)
# school_summary verified output

In [8]:
# Calculating Percent Passing Math and adding column
passing_math_groupby = complete_school_data.loc[complete_school_data.math_score >= 70, :].groupby("school_name").count()
pct_math_passing = (passing_math_groupby.math_score/school_summary['size'])*100
school_summary['% Passing Math'] = pct_math_passing

In [9]:
# Calculating Percent Passing Reading and adding column
passing_reading_groupby = complete_school_data.loc[complete_school_data.reading_score >= 70, :].groupby("school_name").count()
pct_reading_passing = (passing_reading_groupby.reading_score/school_summary['size'])*100
school_summary['% Passing Reading'] = pct_reading_passing

In [10]:
# Calculating Percent Overall Passing and adding column
passing_overall_groupby = complete_school_data.loc[(complete_school_data.math_score >= 70) &
                                                   (complete_school_data.reading_score >= 70),:].groupby("school_name").count()
pct_overall_passing = (passing_overall_groupby.grade/school_summary['size'])*100
school_summary['% Overall Passing'] = pct_overall_passing
# school_summary verified output

In [11]:
# Confirming data types prior to formatting
# school_summary.info()

In [12]:
# Formatting and renaming columns of School Summary Table
school_summary['size'] = school_summary['size'].map('{:,.0f}'.format)
school_summary['budget'] = school_summary['budget'].map('${:,.2f}'.format)
school_summary['Budget per Student'] = school_summary['Budget per Student'].map('${:,.2f}'.format)
school_summary.columns = ['Type', 'Total Students', 'Total Budget', 'Budget per Student',
                          'Average Math Score', 'Average Reading Score', '% Passing Math',
                         'Percent Passing Reading', 'Percent Overall Passing'
                         ]
school_summary

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,Percent Passing Reading,Percent Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Top Performing Schools (By % Overall Passing)

In [13]:
# Sort and display the top five performing schools by % overall passing.
top_performing_schools = school_summary.sort_values('Percent Overall Passing', ascending=False)
top_performing_schools.drop(top_performing_schools.tail(10).index, inplace = True)
top_performing_schools

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,Percent Passing Reading,Percent Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [14]:
# Sort and display the five worst-performing schools by % overall passing.
bottom_perfoming_schools = school_summary.sort_values('Percent Overall Passing')
bottom_perfoming_schools.drop(bottom_perfoming_schools.tail(10).index, inplace = True)
bottom_perfoming_schools

,Type,Total Students,Total Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,Percent Passing Reading,Percent Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [15]:
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
math_scores_by_grade = complete_school_data.groupby(['school_name', 'grade'])['math_score'].mean()

# Using .unstack to move grade index to columns and create dataframe
math_grade = math_scores_by_grade.unstack(level=-1)

#Rearranging columns to improve readability
math_grade = math_grade[['9th', '10th', '11th', '12th']]
math_grade

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Scores by Grade